# Multi-agent Collaboration for Trending Stock Financial Analysis



In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [ ]:
!pip install crewai

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from google.colab import userdata

# Get API keys from Google Colab secrets
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

In [ ]:
import os
from openai import OpenAI

# Initialize the OpenAI client with your API key
client = OpenAI(api_key=openai_api_key)

# Test the API key by making a simple API call
try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Can you verify if my OpenAI API key is working?"},
        ]
    )

    print("API Key is working!")
    print("Response:", response.choices[0].message.content)

except Exception as e:
    print("An error occurred:", str(e))

API Key is working!
Response: I'm unable to directly test your OpenAI API key. However, you can easily verify if your OpenAI API key is working by making a simple API request using it. If the request is successful and you receive the expected response, then your API key is working correctly. If you encounter any issues, you may need to troubleshoot your API key configuration or contact OpenAI support for further assistance.


## crewAI Tools

In [ ]:
!pip install crewai_tools

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Creating Agents

In [ ]:
stock_picker_agent = Agent(
    role="Portfolio Stock Selector",
    goal="Select 5-7 diverse stocks from different sectors to create a balanced portfolio",
    backstory="Expert portfolio manager specializing in identifying high-potential stocks "
    "across various sectors for diversified portfolio construction. Uses market analysis, "
    "momentum indicators, and sector rotation strategies to select complementary stocks.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

# MODIFIED Stock Picker Task with explicit portfolio output
stock_picker_task = Task(
    description=(
        "Analyze current market trends and select exactly 5-7 stocks from DIFFERENT sectors. "
        "Include at least: 1 tech stock, 1 healthcare stock, 1 financial stock, 1 consumer stock. "
        "Consider market cap diversity (mix of large, mid, small cap). "
        "Output format MUST be: 'TICKER1,TICKER2,TICKER3,TICKER4,TICKER5' (comma-separated, no spaces)."
    ),
    expected_output=(
        "Exactly 5-7 stock tickers in comma-separated format like: NVDA,JPM,JNJ,AMZN,XOM "
        "Include brief reason for each selection and sector classification."
    ),
    agent=stock_picker_agent,
)

# Agent in LangChain

from langchain.agents import Tool, AgentExecutor, initialize_agent
from langchain import OpenAI

# Define the tools (assuming these are defined elsewhere)
search_tool = Tool(
    name="search_tool",
    func=your_search_function,  # Define your search function
    description="Tool for searching market data."
)

scrape_tool = Tool(
    name="scrape_tool",
    func=your_scrape_function,  # Define your scraping function
    description="Tool for scraping market trends."
)


# Initialize the language model
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.7)

# Define the agent that uses these tools
stock_picker_agent = initialize_agent(
    tools=[search_tool, scrape_tool],
    llm=llm,
    agent_type="zero-shot-react-description",  # This assumes the agent makes decisions on the fly
    verbose=True
)

# Function to run the stock picking task
def stock_picker_task():
    prompt = "Analyze the market and select the most trending stock ticker for trading."
    response = stock_picker_agent.run(prompt)
    return response




In [ ]:

# data_analyst_agent = Agent(
#     role="Senior Quantitative Data Analyst",
#     goal="Continuously monitor, process, and analyze vast amounts of financial market \
#     data in real-time to uncover actionable insights and predict market movements with precision.",
#     backstory="With deep expertise in quantitative finance and algorithmic trading, this agent is"
#     "equipped with advanced statistical modeling, machine learning, and AI-driven techniques."
#     "The Senior Quantitative Data Analyst is an indispensable asset, integrating historical data"
#     "with real-time analytics to predict trends, assess risks, and support strategic trading decisions."
#     "This agent’s unparalleled ability to interpret complex data sets and automate pattern recognition"
#     "makes it a vital resource for maximizing trading performance.",
#     verbose=True,
#     allow_delegation=True,
#     tools=[scrape_tool, search_tool]
# )





# # Task for Data Analyst Agent: Analyze Market Data

# data_analysis_task = Task(
#     description=(
#         "Continuously monitor and analyze market data for "
#         "the selected stock ({stock_selection}). "
#         "Use statistical modeling and machine learning to "
#         "identify trends and predict market movements."
#     ),
#     expected_output=(
#         "Insights and alerts about significant market "
#         "opportunities or threats for {stock_selection}."
#     ),
#     agent=data_analyst_agent,
# )






# trading_strategy_agent = Agent(
#     role="Lead Trading Strategy Architect",
#     goal="Design, optimize, and validate advanced trading strategies tailored to market insights and"
#     "user-specific criteria.",
#     backstory="As the Lead Trading Strategy Architect, this agent is a mastermind in the development of"
#     "sophisticated trading algorithms. Drawing from extensive expertise in quantitative finance, risk management,"
#     "and algorithmic trading, it systematically designs and iterates on strategies that balance profitability with"
#     "risk. By leveraging insights from the Data Analyst Agent and real-time market conditions,"
#     "this agent ensures that each strategy is both innovative and resilient in the face of market volatility.",
#     verbose=True,
#     allow_delegation=True,
#     tools=[scrape_tool, search_tool]
# )


# # Task for Lead Trading Strategy Architect: Design and Optimize Trading Strategies
# strategy_development_task = Task(
#     description=(
#         "Leverage insights from the Data Analyst Agent and user-defined parameters, such as risk tolerance "
#         "({risk_tolerance}) and trading strategy preference ({trading_strategy_preference}), to design, optimize,"
#         " and validate a robust set of trading strategies. Continuously refine these strategies based on "
#         "backtesting results, market feedback, and evolving financial conditions."
#     ),
#     expected_output=(
#         "A portfolio of high-potential trading strategies for {stock_selection} that are rigorously tested,"
#         "optimized for performance, and aligned with the user's risk and preference profiles."
#     ),
#     agent=trading_strategy_agent,
# )




# execution_agent = Agent(
#     role="Chief Trade Execution Strategist",
#     goal="Design and recommend precise trade execution strategies that optimize timing, pricing, and market impact based on the approved trading strategies.",
#     backstory="As the Chief Trade Execution Strategist, this agent excels in the art and science of trade execution. With a keen understanding of market dynamics, order types, and execution logistics, the agent meticulously analyzes each trade's potential impact and timing. By considering factors such as liquidity, volatility, and slippage, the agent ensures that every trade is executed with maximum efficiency and adherence to the overarching trading strategy, minimizing costs and optimizing outcomes.",
#     verbose=True,
#     allow_delegation=True,
#     tools=[scrape_tool, search_tool]
# )


# # Task for Chief Trade Execution Strategist: Formulate Trade Execution Plan
# execution_planning_task = Task(
#     description=(
#         "Examine the approved trading strategies and devise precise execution methods for {stock_selection}. "
#         "Consider current market conditions, liquidity, volatility, and optimal pricing to formulate the most effective trade execution plan. "
#         "Ensure that the plan minimizes market impact and aligns with the strategic objectives."
#     ),
#     expected_output=(
#         "A comprehensive and actionable execution plan detailing the optimal timing, order types, and pricing for executing trades in {stock_selection}, ensuring maximum efficiency and alignment with the approved trading strategies."
#     ),
#     agent=execution_agent,
# )



# risk_management_agent = Agent(
#     role="Chief Risk Architect",
#     goal="Conduct in-depth evaluations and provide strategic insights on the risks associated with potential trading activities, ensuring alignment with the firm’s risk management framework.",
#     backstory="As the Chief Risk Architect, this agent is a seasoned expert in risk assessment models and market dynamics. With a sharp focus on identifying and mitigating risk, it rigorously analyzes proposed trades to uncover hidden vulnerabilities. The agent delivers comprehensive risk assessments and recommends actionable safeguards to keep trading activities within acceptable risk parameters.",
#     verbose=True,
#     allow_delegation=True,
#     tools=[scrape_tool, search_tool]
# )


# # Task for Chief Risk Architect: Comprehensive Risk Assessment
# risk_assessment_task = Task(
#     description=(
#         "Perform a thorough evaluation of the risks tied to the proposed trading "
#         "strategies and execution plans for {stock_selection}. "
#         "Deliver a detailed analysis highlighting potential risks and "
#         "propose robust mitigation strategies to minimize exposure."
#     ),
#     expected_output=(
#         "A detailed risk assessment report that outlines potential risks "
#         "and provides strategic recommendations for risk mitigation related to {stock_selection}."
#     ),
#     agent=risk_management_agent,
# )



In [ ]:
import os
from crewai import Agent, Task, Crew, Process

# Your existing agents remain unchanged
data_analyst_agent = Agent(
    role="Senior Quantitative Data Analyst",
    goal="Continuously monitor, process, and analyze vast amounts of financial market \
    data in real-time to uncover actionable insights and predict market movements with precision.",
    backstory="With deep expertise in quantitative finance and algorithmic trading, this agent is"
    "equipped with advanced statistical modeling, machine learning, and AI-driven techniques."
    "The Senior Quantitative Data Analyst is an indispensable asset, integrating historical data"
    "with real-time analytics to predict trends, assess risks, and support strategic trading decisions."
    "This agent's unparalleled ability to interpret complex data sets and automate pattern recognition"
    "makes it a vital resource for maximizing trading performance.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

# Add this reminder to each task description
reminder = "IMPORTANT: Analyze ALL stocks in the portfolio ({stock_selection}), not just the first one. "

# Example for data_analysis_task
data_analysis_task = Task(
    description=(
        reminder +
        "Monitor and analyze market data for EACH stock in portfolio: {stock_selection}. "
        "Provide individual analysis for each stock AND portfolio-level insights."
    ),
    expected_output=(
        "Analysis for EACH stock in {stock_selection} including: "
        "1) Individual performance metrics for all stocks, "
        "2) Correlation matrix between all stocks, "
        "3) Portfolio-level opportunities and risks."
    ),
    agent=data_analyst_agent,
)

trading_strategy_agent = Agent(
    role="Lead Trading Strategy Architect",
    goal="Design, optimize, and validate advanced trading strategies tailored to market insights and"
    "user-specific criteria.",
    backstory="As the Lead Trading Strategy Architect, this agent is a mastermind in the development of"
    "sophisticated trading algorithms. Drawing from extensive expertise in quantitative finance, risk management,"
    "and algorithmic trading, it systematically designs and iterates on strategies that balance profitability with"
    "risk. By leveraging insights from the Data Analyst Agent and real-time market conditions,"
    "this agent ensures that each strategy is both innovative and resilient in the face of market volatility.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

# Modified task to handle portfolio
strategy_development_task = Task(
    description=(
        "Design portfolio strategies for stocks: {stock_selection}. Create individual strategies "
        "per stock AND portfolio-level strategies. Consider allocation weights, rebalancing, "
        "and correlation-based opportunities. Risk tolerance: {risk_tolerance}, "
        "Strategy: {trading_strategy_preference}."
    ),
    expected_output=(
        "Portfolio strategy including: 1) Allocation % for each stock in {stock_selection}, "
        "2) Individual strategies per stock, 3) Portfolio-level strategies, "
        "4) Expected portfolio returns and Sharpe ratio."
    ),
    agent=trading_strategy_agent,
)

execution_agent = Agent(
    role="Chief Trade Execution Strategist",
    goal="Design and recommend precise trade execution strategies that optimize timing, pricing, and market impact based on the approved trading strategies.",
    backstory="As the Chief Trade Execution Strategist, this agent excels in the art and science of trade execution. With a keen understanding of market dynamics, order types, and execution logistics, the agent meticulously analyzes each trade's potential impact and timing. By considering factors such as liquidity, volatility, and slippage, the agent ensures that every trade is executed with maximum efficiency and adherence to the overarching trading strategy, minimizing costs and optimizing outcomes.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

# Modified task to handle portfolio
execution_planning_task = Task(
    description=(
        "Create execution plans for portfolio {stock_selection}. Design execution sequence, "
        "order sizes based on allocations, and timing to minimize total market impact. "
        "Consider executing correlated stocks separately."
    ),
    expected_output=(
        "Portfolio execution plan: 1) Execution order and timing for each stock, "
        "2) Position sizes based on allocation, 3) Total capital deployment schedule, "
        "4) Market impact minimization strategy."
    ),
    agent=execution_agent,
)

risk_management_agent = Agent(
    role="Chief Risk Architect",
    goal="Conduct in-depth evaluations and provide strategic insights on the risks associated with potential trading activities, ensuring alignment with the firm's risk management framework.",
    backstory="As the Chief Risk Architect, this agent is a seasoned expert in risk assessment models and market dynamics. With a sharp focus on identifying and mitigating risk, it rigorously analyzes proposed trades to uncover hidden vulnerabilities. The agent delivers comprehensive risk assessments and recommends actionable safeguards to keep trading activities within acceptable risk parameters.",
    verbose=True,
    allow_delegation=True,
    tools=[scrape_tool, search_tool]
)

# Modified task to handle portfolio
risk_assessment_task = Task(
    description=(
        "Assess risks for portfolio {stock_selection}. Analyze individual stock risks "
        "AND portfolio risks: concentration, correlation, sector exposure. "
        "Calculate portfolio VaR, max drawdown, and beta."
    ),
    expected_output=(
        "Portfolio risk report: 1) Individual stock risks, 2) Portfolio VaR and beta, "
        "3) Correlation risks, 4) Recommended position limits and hedges for {stock_selection}."
    ),
    agent=risk_management_agent,
)

# When running, use comma-separated stocks:
# financial_trading_inputs['stock_selection'] = 'NVDA,AAPL,MSFT,GOOGL,AMZN'

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [ ]:
# from crewai import Crew, Process
# from langchain_openai import ChatOpenAI




# stock_picker_crew = Crew(
#     agents=[stock_picker_agent],

#     tasks=[stock_picker_task],

#     manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
#                            temperature=0.7),
#     process=Process.sequential,  # Change here if needed
#     verbose=True
# )


# financial_trading_crew = Crew(
#     agents=[stock_picker_agent,
#             data_analyst_agent,
#             trading_strategy_agent,
#             execution_agent,
#             risk_management_agent],

#     tasks=[stock_picker_task,
#            data_analysis_task,
#            strategy_development_task,
#            execution_planning_task,
#            risk_assessment_task],

#     manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
#                            temperature=0.7),
#     process=Process.sequential,  # Change here if needed
#     #process=Process.hierarchical,
#     verbose=True
# )

from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Stock picker crew remains the same
stock_picker_crew = Crew(
    agents=[stock_picker_agent],
    tasks=[stock_picker_task],
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.sequential,
    verbose=True
)

# MODIFIED: Remove stock_picker from financial_trading_crew
financial_trading_crew = Crew(
    agents=[
        data_analyst_agent,
        trading_strategy_agent,
        execution_agent,
        risk_management_agent
    ],
    tasks=[
        data_analysis_task,
        strategy_development_task,
        execution_planning_task,
        risk_assessment_task
    ],
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo",
                           temperature=0.7),
    process=Process.sequential,
    verbose=True
)


# Using langchain orchestraition

# Define tasks as functions
def run_data_analysis_task(stock_selection):
    prompt = f"Analyze market data for {stock_selection} using statistical modeling."
    analysis_result = data_analyst_agent.run(prompt)
    return analysis_result

# Orchestrate the agents
def run_financial_trading_process(initial_inputs):
    # Step 1: Stock Picker Task
    stock_selection = run_stock_picker_task()
    print(f"Selected stock: {stock_selection}")
    
    # Step 2: Data Analysis Task
    analysis_result = run_data_analysis_task(stock_selection)
    print(f"Data analysis result: {analysis_result}")
    
    # Continue with other tasks...
    # Example: run_trading_strategy_task(), run_execution_task(), etc.

    return {
        "stock_selection": stock_selection,
        "analysis_result": analysis_result,
        # Add other results here
    }

# Run the financial trading process
initial_inputs = {
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

results = run_financial_trading_process(initial_inputs)
print(results)

## Running the Crew

- Set the inputs for the execution of the crew.

In [ ]:
# Initialize the process with only the initial inputs
financial_trading_inputs = {
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

# Run the process
#financial_trading_crew.run(inputs=financial_trading_inputs)

# After the Stock Picker Agent picks a stock, it will dynamically set 'stock_selection'


**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [ ]:
# MODIFIED Stock Picker Agent to explicitly return multiple stocks
stock_picker_agent = Agent(
    role="Portfolio Stock Selector",
    goal="Select 5-7 diverse stocks from different sectors to create a balanced portfolio",
    backstory="Expert portfolio manager specializing in identifying high-potential stocks "
    "across various sectors for diversified portfolio construction. Uses market analysis, "
    "momentum indicators, and sector rotation strategies to select complementary stocks.",
    verbose=True,
    allow_delegation=False,
    tools=[search_tool, scrape_tool]
)

# MODIFIED Stock Picker Task with explicit portfolio output
stock_picker_task = Task(
    description=(
        "Analyze current market trends and select exactly 5-7 stocks from DIFFERENT sectors. "
        "Include at least: 1 tech stock, 1 healthcare stock, 1 financial stock, 1 consumer stock. "
        "Consider market cap diversity (mix of large, mid, small cap). "
        "Output format MUST be: 'TICKER1,TICKER2,TICKER3,TICKER4,TICKER5' (comma-separated, no spaces)."
    ),
    expected_output=(
        "Exactly 5-7 stock tickers in comma-separated format like: NVDA,JPM,JNJ,AMZN,XOM "
        "Include brief reason for each selection and sector classification."
    ),
    agent=stock_picker_agent,
)

# Also update the portfolio extraction function to be more robust
def extract_tickers_from_output(crew_output):
    """Extract multiple tickers from crew output"""
    raw_text = crew_output.raw

    # First try to find comma-separated list
    import re

    # Look for pattern like "AAPL,MSFT,GOOGL" or "AAPL, MSFT, GOOGL"
    pattern = r'([A-Z]{1,5}(?:\s*,\s*[A-Z]{1,5}){3,6})'
    match = re.search(pattern, raw_text)

    if match:
        tickers_str = match.group(1)
        tickers = [t.strip() for t in tickers_str.split(',')]
        return tickers

    # If no comma-separated list, find all tickers mentioned
    all_tickers = re.findall(r'\b([A-Z]{2,5})\b', raw_text)

    # Filter out common non-ticker words
    exclude = {'CEO', 'CFO', 'IPO', 'ETF', 'NYSE', 'NASDAQ', 'USD', 'API', 'USA', 'FDA', 'SEC'}
    tickers = []
    seen = set()

    for ticker in all_tickers:
        if ticker not in exclude and ticker not in seen and len(ticker) <= 5:
            tickers.append(ticker)
            seen.add(ticker)
            if len(tickers) >= 7:  # Limit to 7 stocks
                break

    # If we found less than 3 stocks, there's likely an issue
    if len(tickers) < 3:
        print(f"Warning: Only found {len(tickers)} tickers. Adding default diversified portfolio.")
        # Return a default diversified portfolio
        return ['NVDA', 'AAPL', 'JPM', 'JNJ', 'XOM']

    return tickers

# When running the crew, ensure the portfolio is properly formatted
stock_picker_result = stock_picker_crew.kickoff()
portfolio_tickers = extract_tickers_from_output(stock_picker_result)

print(f"Selected Portfolio: {portfolio_tickers}")
print(f"Number of stocks: {len(portfolio_tickers)}")

# Make sure to join with comma AND space for better readability
financial_trading_inputs['stock_selection'] = ', '.join(portfolio_tickers)
print(f"Passing to crew: {financial_trading_inputs['stock_selection']}")

result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: ea1d01b2-0857-4ddb-b897-1b9f787529c5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Portfolio Stock Selector                                                                                │
│                                                                                                                 │
│  Task: Analyze current market trends and select exactly 5-7 stocks from DIFFERENT sectors. Include at least: 1  │
│  tech stock, 1 healthcare stock, 1 financial stock, 1 consumer stock. Consider market cap diversity (mix of     │
│  large, mid, small cap). Output format MUST be: 'TICKER1,TICKER2,TICKER3,TICKER4,TICKER5' (comma-separated, no  │
│  spaces).                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:112: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  self.msg_id = ip.kernel._parent_header['header']['msg_id']

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'ADBE stock analysis', 'type': 'search', 'num': 10, 'engine': 'google'},            │
│  'organic': [{'title': 'Adobe (ADBE) Stock Price & Overview', 'link':                                           │
│  'https://stockanalysis.com/stocks/adbe/', 'snippet': 'Analyst Summary\u200b\u200b According to 22 analysts,    │
│  the average rating for ADBE stock is "Buy." The 12-month stock price target is $490.2, which is an increase    │
│  of 35.50% ...', 'position': 1, 'sitelinks': [{'title': 'Forecast', 'link':                                     │
│  'https://stockanalysis.com/stocks/adbe/forecast/'}, {'title': 'Statistics', 'link':                            │
│  'https://stockanalysis.com/stocks/adbe/statistics/'}, {'title': 'Profile', 'link':                             │
│  'https://stockanalysis.com/stocks/adbe/company/'}, {'title': 'Adobe Dividend Information', 'link':             │
│  'https://stockanalysis.com/stocks/adbe/dividend/'}]}, {'title': 'Adobe Inc. (ADBE) - Yahoo Finance', 'link':   │
│  'https://finance.yahoo.com/quote/ADBE/', 'snippet': '$ADBE: Designing a Creative Empire ; FCF/Share CAGR       │
│  10-12 % (HSD revenue growth, continued margin expansion, 1.5% share reduction). ; Terminal multiple 25× (near  │
│  ...', 'position': 2, 'sitelinks': [{'title': '361.77', 'link': 'https://ca.finance.yahoo.com/quote/ADBE/'},    │
│  {'title': '360.81', 'link': 'https://sg.finance.yahoo.com/quote/ADBE/'}, {'title': 'Meta_title_quote',         │
│  'link': 'https://finance.yahoo.com/quote/ADBE/news/'}, {'title': 'Historical Data', 'link':                    │
│  'https://finance.yahoo.com/quote/ADBE/history/'}]}, {'title': 'Adobe (ADBE) Stock Forecast, Price Targets and  │
│  Analysts Predictions', 'link': 'https://www.tipranks.com/stocks/adbe/forecast', 'snippet': 'In the current     │
│  month, ADBE has received 29 Buy Ratings, 12 Hold Ratings, and 2 Sell Ratings. ADBE average Analyst price       │
│  target in the past 3 months is 484.88.', 'position': 3}, {'title': 'ADBE Stock Quote Price and Forecast |      │
│  CNN', 'link': 'https://www.cnn.com/markets/stocks/ADBE', 'snippet': 'Smart score. ADBE has a strong chance to  │
│  outperform the market over the next 12 months.', 'position': 4}, {'title': 'ADBE Stock Price and Chart - A...  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Chief Risk Architect                                                                                    │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Portfolio risk report:                                                                                         │
│  1) **Individual Stock Risks**:                                                                                 │
│     - DAVA: Endava plc (DAVA) stock has a potential increase of 94.97% with an average rating of "Buy."         │
│     - AKAM: Akamai Technologies, Inc. (AKAM) stock indicates a potential increase of 36.49% with an average     │
│  rating of "Buy."                                                                                               │
│     - SENS: Senseonics Holdings, Inc. (SENS) stock shows a potential increase of 262.25% with a rating of       │
│  "Strong Buy."                                                                                                  │
│     - NICE: NICE Ltd. (NICE) stock presents a potential increase of 38.14% with a rating of "Buy."              │
│     - SABR: Sabre Corporation (SABR) stock implies a potential increase of 34.86% with an average rating of     │
│  "Hold."                                                                                                        │
│     - GLOB: Globant (GLOB) stock suggests a potential increase of 109.21% with a rating of "Buy."               │
│     - ADBE: Adobe Inc. (ADBE) stock indicates a potential increase of 35.50% with a rating of "Buy."            │
│                                                                                                                 │
│  2) **Portfolio VaR and Beta**:                                                                                 │
│     - VaR, max drawdown, and beta calculations are pending based on correlation and individual risk analysis.   │
│                                                                                                                 │
│  3) **Correlation Risks**:                                                                                      │
│     - Detailed analysis of correlation coefficients between the stock prices of DAVA, AKAM, SENS, NICE, SABR,   │
│  GLOB, ADBE is needed to assess correlation risks.                                                              │
│                                                                                                                 │
│  4) **Recommended Position Limits and Hedges**:                                                                 │
│     - Position limits and hedges recommendations will be based on the individual stock risks, correlation       │
│  analysis, and the portfolio's overall risk exposure.                                                           │
│                                                                                                                 │
│  I will now proceed with the calculation of portfolio VaR, max drawdown, beta, and further analysis to provide  │
│  a comprehensive risk assessment for the portfolio.                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0f73e576-0b0f-49fa-8578-977a392a00b1                                                                     │
│  Agent: Chief Risk Architect                                                                                    │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 559b3892-9044-44ee-b3f2-e2eaf4d0b2f4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Portfolio risk report:                                                                           │
│  1) **Individual Stock Risks**:                                                                                 │
│     - DAVA: Endava plc (DAVA) stock has a potential increase of 94.97% with an average rating of "Buy."         │
│     - AKAM: Akamai Technologies, Inc. (AKAM) stock indicates a potential increase of 36.49% with an average     │
│  rating of "Buy."                                                                                               │
│     - SENS: Senseonics Holdings, Inc. (SENS) stock shows a potential increase of 262.25% with a rating of       │
│  "Strong Buy."                                                                                                  │
│     - NICE: NICE Ltd. (NICE) stock presents a potential increase of 38.14% with a rating of "Buy."              │
│     - SABR: Sabre Corporation (SABR) stock implies a potential increase of 34.86% with an average rating of     │
│  "Hold."                                                                                                        │
│     - GLOB: Globant (GLOB) stock suggests a potential increase of 109.21% with a rating of "Buy."               │
│     - ADBE: Adobe Inc. (ADBE) stock indicates a potential increase of 35.50% with a rating of "Buy."            │
│                                                                                                                 │
│  2) **Portfolio VaR and Beta**:                                                                                 │
│     - VaR, max drawdown, and beta calculations are pending based on correlation and individual risk analysis.   │
│                                                                                                                 │
│  3) **Correlation Risks**:                                                                                      │
│     - Detailed analysis of correlation coefficients between the stock prices of DAVA, AKAM, SENS, NICE, SABR,   │
│  GLOB, ADBE is needed to assess correlation risks.                                                              │
│                                                                                                                 │
│  4) **Recommended Position Limits and Hedges**:                                                                 │
│     - Position limits and hedges recommendations will be based on the individual stock risks, correlation       │
│  analysis, and the portfolio's overall risk exposure.                                                           │
│                                                                                                                 │
│  I will now proceed with the calculation of portfolio VaR, max drawdown, beta, and further analysis to provide  │
│  a comprehensive risk assessment for the portfolio.                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰───────────────────────────────────────────────────────

- Display the final result as Markdown.

In [ ]:
result

CrewOutput(raw='Portfolio risk report:\n1) **Individual Stock Risks**:\n   - DAVA: Endava plc (DAVA) stock has a potential increase of 94.97% with an average rating of "Buy."\n   - AKAM: Akamai Technologies, Inc. (AKAM) stock indicates a potential increase of 36.49% with an average rating of "Buy."\n   - SENS: Senseonics Holdings, Inc. (SENS) stock shows a potential increase of 262.25% with a rating of "Strong Buy."\n   - NICE: NICE Ltd. (NICE) stock presents a potential increase of 38.14% with a rating of "Buy."\n   - SABR: Sabre Corporation (SABR) stock implies a potential increase of 34.86% with an average rating of "Hold."\n   - GLOB: Globant (GLOB) stock suggests a potential increase of 109.21% with a rating of "Buy."\n   - ADBE: Adobe Inc. (ADBE) stock indicates a potential increase of 35.50% with a rating of "Buy."\n\n2) **Portfolio VaR and Beta**: \n   - VaR, max drawdown, and beta calculations are pending based on correlation and individual risk analysis.\n\n3) **Correlation R

In [ ]:
def print_portfolio_analysis(crew_output):
    """Print a comprehensive, beautifully formatted portfolio analysis"""

    # Extract stock data from the analysis
    import re

    # Parse individual stock data
    stock_data = {}
    raw_text = crew_output.tasks_output[0].raw  # Data analyst output

    # Pattern to extract stock info
    patterns = {
        'DAVA': r'DAVA.*?trading at.*?\$(\d+\.\d+).*?target.*?\$(\d+\.\d+).*?increase of (\d+\.\d+)%',
        'AKAM': r'AKAM.*?target.*?\$(\d+\.\d+).*?increase of (\d+\.\d+)%',
        'SENS': r'SENS.*?trading at.*?\$(\d+\.\d+).*?target.*?\$(\d+\.\d+).*?increase of (\d+\.\d+)%',
        'NICE': r'NICE.*?priced at \$(\d+\.\d+).*?target.*?\$(\d+\.\d+).*?increase of (\d+\.\d+)%',
        'SABR': r'SABR.*?price is.*?\$(\d+\.\d+).*?target.*?\$(\d+\.\d+).*?increase of (\d+\.\d+)%',
        'GLOB': r'GLOB.*?trading at \$(\d+\.\d+).*?target.*?\$(\d+\.\d+).*?increase of (\d+\.\d+)%',
        'ADBE': r'ADBE.*?priced at \$(\d+\.\d+).*?target.*?\$(\d+\.\d+).*?increase of (\d+\.\d+)%'
    }

    # Extract execution order
    exec_text = crew_output.tasks_output[2].raw
    exec_order = ['SENS', 'DAVA', 'AKAM', 'GLOB', 'NICE', 'ADBE', 'SABR']

    print("\n" + "="*100)
    print("                        📊 PORTFOLIO ANALYSIS REPORT                         ")
    print("="*100)

    # Portfolio Overview
    portfolio_tickers = ['DAVA', 'AKAM', 'SENS', 'NICE', 'SABR', 'GLOB', 'ADBE']
    print("\n🎯 PORTFOLIO COMPOSITION: " + " • ".join(portfolio_tickers))
    print(f"📈 Total Positions: {len(portfolio_tickers)}")
    print(f"⚙️  Strategy: Day Trading | Risk Tolerance: Medium")

    # Individual Stock Analysis
    print("\n" + "="*100)
    print("                        📋 INDIVIDUAL STOCK ANALYSIS                         ")
    print("="*100)

    # Parse ratings and targets from the risk report
    risk_text = crew_output.raw

    stock_info = {
        'DAVA': {'company': 'Endava plc', 'rating': 'Buy', 'potential': '94.97%'},
        'AKAM': {'company': 'Akamai Technologies', 'rating': 'Buy', 'potential': '36.49%'},
        'SENS': {'company': 'Senseonics Holdings', 'rating': 'Strong Buy', 'potential': '262.25%'},
        'NICE': {'company': 'NICE Ltd.', 'rating': 'Buy', 'potential': '38.14%'},
        'SABR': {'company': 'Sabre Corporation', 'rating': 'Hold', 'potential': '34.86%'},
        'GLOB': {'company': 'Globant', 'rating': 'Buy', 'potential': '109.21%'},
        'ADBE': {'company': 'Adobe Inc.', 'rating': 'Buy', 'potential': '35.50%'}
    }

    print("\n{:<6} {:<25} {:<12} {:<15}".format("Ticker", "Company", "Rating", "Potential Gain"))
    print("-" * 70)

    for ticker, info in stock_info.items():
        rating_emoji = "🟢" if "Strong Buy" in info['rating'] else "🟡" if "Buy" in info['rating'] else "🟠"
        print("{:<6} {:<25} {} {:<10} {:<15}".format(
            ticker,
            info['company'][:24],
            rating_emoji,
            info['rating'],
            info['potential']
        ))

    # Top Opportunities
    print("\n" + "="*100)
    print("                        🚀 TOP OPPORTUNITIES                                ")
    print("="*100)

    opportunities = [
        ("SENS", "262.25%", "Strong Buy - Highest potential return"),
        ("GLOB", "109.21%", "Buy - Strong growth potential"),
        ("DAVA", "94.97%", "Buy - Near doubling opportunity")
    ]

    for i, (ticker, gain, reason) in enumerate(opportunities, 1):
        print(f"\n{i}. {ticker} - Expected Gain: {gain}")
        print(f"   📍 {reason}")

    # Execution Strategy
    print("\n" + "="*100)
    print("                        ⚡ EXECUTION STRATEGY                               ")
    print("="*100)

    print("\n📅 Recommended Execution Sequence (optimized for market impact):")
    for i, ticker in enumerate(exec_order, 1):
        priority = "🔴 HIGH" if ticker in ['SENS', 'DAVA', 'GLOB'] else "🟡 MEDIUM" if ticker in ['AKAM', 'NICE', 'ADBE'] else "🟢 LOW"
        print(f"   {i}. {ticker} - Priority: {priority}")

    print("\n💡 Execution Tips:")
    print("   • Use limit orders to minimize slippage")
    print("   • Deploy capital gradually over the trading day")
    print("   • Execute correlated stocks separately")
    print("   • Monitor real-time market conditions")

    # Risk Assessment
    print("\n" + "="*100)
    print("                        ⚠️  RISK ASSESSMENT                                 ")
    print("="*100)

    print("\n🛡️ Risk Mitigation Strategies:")
    print("   • Portfolio VaR and Beta calculations pending")
    print("   • Correlation analysis required between all positions")
    print("   • Position sizing based on individual stock volatility")
    print("   • Stop-loss orders recommended for all positions")

    print("\n📊 Risk Categories:")
    print("   • High Risk/High Reward: SENS (262% potential)")
    print("   • Medium Risk: DAVA, GLOB, AKAM, NICE, ADBE")
    print("   • Lower Risk: SABR (Hold rating)")

    # Summary Statistics
    print("\n" + "="*100)
    print("                        📈 PORTFOLIO SUMMARY                                ")
    print("="*100)

    # Calculate average potential
    potentials = [94.97, 36.49, 262.25, 38.14, 34.86, 109.21, 35.50]
    avg_potential = sum(potentials) / len(potentials)

    print(f"\n💰 Average Portfolio Potential: {avg_potential:.2f}%")
    print(f"🎯 Buy Recommendations: 6 out of 7 stocks")
    print(f"⭐ Strong Buy Recommendations: 1 (SENS)")
    print(f"📊 Sectors: Technology, Software, Healthcare Tech")

    # Token Usage
    print("\n" + "="*100)
    print("                        🔧 ANALYSIS METRICS                                 ")
    print("="*100)
    print(f"\n📊 Total Tokens Used: {crew_output.token_usage.total_tokens:,}")
    print(f"🤖 AI Requests: {crew_output.token_usage.successful_requests}")
    print(f"⏱️  Agents Involved: 4 (Data Analyst, Strategy Architect, Execution Strategist, Risk Architect)")

    print("\n" + "="*100)
    print("                    💡 NEXT STEPS                                          ")
    print("="*100)
    print("\n1. Review individual stock fundamentals")
    print("2. Set position sizes based on risk tolerance")
    print("3. Place initial orders following execution sequence")
    print("4. Monitor positions throughout the trading day")
    print("5. Implement stop-loss and take-profit levels")

    print("\n" + "="*100 + "\n")

# Call the function
print_portfolio_analysis(result)

# Additional concise summary
def print_quick_summary(crew_output):
    """Print a quick executive summary"""
    print("\n🎯 EXECUTIVE SUMMARY")
    print("=" * 50)
    print("Portfolio: DAVA, AKAM, SENS, NICE, SABR, GLOB, ADBE")
    print("Top Pick: SENS (262.25% potential)")
    print("Strategy: Day Trading with Medium Risk")
    print("Action: Execute in order - SENS → DAVA → AKAM → GLOB")
    print("=" * 50)

print_quick_summary(result)


                        📊 PORTFOLIO ANALYSIS REPORT                         

🎯 PORTFOLIO COMPOSITION: DAVA • AKAM • SENS • NICE • SABR • GLOB • ADBE
📈 Total Positions: 7
⚙️  Strategy: Day Trading | Risk Tolerance: Medium

                        📋 INDIVIDUAL STOCK ANALYSIS                         

Ticker Company                   Rating       Potential Gain 
----------------------------------------------------------------------
DAVA   Endava plc                🟡 Buy        94.97%         
AKAM   Akamai Technologies       🟡 Buy        36.49%         
SENS   Senseonics Holdings       🟢 Strong Buy 262.25%        
NICE   NICE Ltd.                 🟡 Buy        38.14%         
SABR   Sabre Corporation         🟠 Hold       34.86%         
GLOB   Globant                   🟡 Buy        109.21%        
ADBE   Adobe Inc.                🟡 Buy        35.50%         

                        🚀 TOP OPPORTUNITIES                                

1. SENS - Expected Gain: 262.25%
   📍 Strong Buy - Hi

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
import re
warnings.filterwarnings('ignore')

def extract_portfolio_from_result(crew_output):
    """Extract stock tickers from CrewOutput result"""

    # Method 1: Try to extract from task descriptions (most reliable)
    for task in crew_output.tasks_output:
        if 'portfolio stocks:' in task.description or 'portfolio' in task.description.lower():
            # Look for pattern like "DAVA, AKAM, SENS, NICE, SABR, GLOB, ADBE"
            match = re.search(r'portfolio\s+(?:stocks:\s+)?([A-Z]{2,5}(?:\s*,\s*[A-Z]{2,5})+)', task.description)
            if match:
                tickers_str = match.group(1)
                tickers = [t.strip() for t in tickers_str.split(',')]
                return tickers

    # Method 2: Extract from raw output
    raw_text = crew_output.raw

    # Look for "Individual Stock Risks" section
    if "Individual Stock Risks" in raw_text:
        # Extract all tickers that appear with a colon after them
        ticker_pattern = r'([A-Z]{2,5}):'
        matches = re.findall(ticker_pattern, raw_text)
        # Remove duplicates while preserving order
        seen = set()
        tickers = []
        for ticker in matches:
            if ticker not in seen and len(ticker) <= 5:
                seen.add(ticker)
                tickers.append(ticker)

        if len(tickers) >= 3:  # Ensure we found a reasonable number
            return tickers

    # Method 3: Extract all valid tickers from the entire output
    all_tickers = re.findall(r'\b([A-Z]{2,5})\b', crew_output.raw)

    # Filter out common non-ticker words
    exclude = {'CEO', 'CFO', 'IPO', 'ETF', 'NYSE', 'USD', 'API', 'USA', 'VaR', 'HOLD', 'BUY', 'SELL'}

    # Count occurrences
    ticker_counts = {}
    for ticker in all_tickers:
        if ticker not in exclude and len(ticker) >= 2:
            ticker_counts[ticker] = ticker_counts.get(ticker, 0) + 1

    # Get most frequent tickers (likely to be the portfolio stocks)
    sorted_tickers = sorted(ticker_counts.items(), key=lambda x: x[1], reverse=True)
    portfolio_tickers = [ticker for ticker, count in sorted_tickers[:10] if count >= 2]

    return portfolio_tickers[:7]  # Return top 7 most mentioned

def backtest_portfolio(tickers, start_date, end_date, initial_capital=100000,
                      weights=None, risk_free_rate=0.02):
    """
    Backtest a portfolio of stocks

    Parameters:
    - tickers: list of stock symbols
    - start_date: start date for backtest (YYYY-MM-DD)
    - end_date: end date for backtest (YYYY-MM-DD)
    - initial_capital: starting capital (default $100,000)
    - weights: portfolio weights (default equal weight)
    - risk_free_rate: annual risk-free rate for Sharpe calculation
    """

    print(f"\n{'='*80}")
    print(f"{'PORTFOLIO BACKTEST ANALYSIS':^80}")
    print(f"{'='*80}")
    print(f"\n📅 Period: {start_date} to {end_date}")
    print(f"💰 Initial Capital: ${initial_capital:,.2f}")
    print(f"📊 Portfolio: {', '.join(tickers)}")

    # Download stock data
    print("\n⏳ Downloading historical data...")
    data = yf.download(tickers, start=start_date, end=end_date, progress=False)
    print (data.head(3))
    data = data['Close']

    # Handle single stock case
    if len(tickers) == 1:
        data = pd.DataFrame(data)
        data.columns = tickers

    # Calculate prices at start and end
    start_prices = data.iloc[0]
    end_prices = data.iloc[-1]

    # Set weights if not provided
    if weights is None:
        weights = np.array([1/len(tickers)] * len(tickers))
    else:
        weights = np.array(weights)

    # Calculate shares to buy for each stock
    position_values = initial_capital * weights
    shares = position_values / start_prices

    print("\n📈 POSITION DETAILS:")
    print("-" * 80)
    print(f"{'Ticker':<8} {'Weight':<10} {'Start Price':<12} {'End Price':<12} {'Shares':<10} {'Position Value':<15}")
    print("-" * 80)

    for i, ticker in enumerate(tickers):
        print(f"{ticker:<8} {weights[i]*100:<9.1f}% ${start_prices[ticker]:<11.2f} ${end_prices[ticker]:<11.2f} {shares[ticker]:<10.2f} ${position_values[i]:<14,.2f}")

    # Calculate daily portfolio value
    portfolio_value = (data * shares).sum(axis=1)
    portfolio_returns = portfolio_value.pct_change().dropna()

    # Calculate individual stock returns
    stock_returns = data.pct_change().dropna()

    # Calculate metrics
    total_return = (portfolio_value.iloc[-1] - initial_capital) / initial_capital
    annualized_return = (1 + total_return) ** (252 / len(portfolio_returns)) - 1
    volatility = portfolio_returns.std() * np.sqrt(252)
    sharpe_ratio = (annualized_return - risk_free_rate) / volatility

    # Calculate maximum drawdown
    cumulative = (1 + portfolio_returns).cumprod()
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()

    # Calculate individual stock performance
    individual_returns = ((end_prices - start_prices) / start_prices).sort_values(ascending=False)

    print("\n" + "="*80)
    print(f"{'PORTFOLIO PERFORMANCE METRICS':^80}")
    print("="*80)

    print(f"\n💵 Final Portfolio Value: ${portfolio_value.iloc[-1]:,.2f}")
    print(f"📊 Total Return: {total_return*100:.2f}%")
    print(f"📈 Annualized Return: {annualized_return*100:.2f}%")
    print(f"📉 Volatility (Annual): {volatility*100:.2f}%")
    print(f"⚖️  Sharpe Ratio: {sharpe_ratio:.3f}")
    print(f"🔻 Maximum Drawdown: {max_drawdown*100:.2f}%")
    print(f"📅 Trading Days: {len(portfolio_returns)}")

    # Best and worst days
    best_day = portfolio_returns.max()
    worst_day = portfolio_returns.min()
    print(f"\n🟢 Best Day: {best_day*100:.2f}% on {portfolio_returns.idxmax().strftime('%Y-%m-%d')}")
    print(f"🔴 Worst Day: {worst_day*100:.2f}% on {portfolio_returns.idxmin().strftime('%Y-%m-%d')}")

    print("\n" + "="*80)
    print(f"{'INDIVIDUAL STOCK PERFORMANCE':^80}")
    print("="*80)
    print(f"\n{'Ticker':<8} {'Return':<12} {'Ann. Return':<15} {'Volatility':<12} {'Contribution':<15}")
    print("-" * 80)

    for ticker in tickers:
        stock_return = individual_returns[ticker]
        ann_return = (1 + stock_return) ** (252 / len(stock_returns)) - 1
        stock_vol = stock_returns[ticker].std() * np.sqrt(252)
        contribution = stock_return * weights[tickers.index(ticker)] * 100

        print(f"{ticker:<8} {stock_return*100:<11.2f}% {ann_return*100:<14.2f}% {stock_vol*100:<11.2f}% {contribution:<14.2f}%")

    # Risk-adjusted metrics
    print("\n" + "="*80)
    print(f"{'RISK-ADJUSTED METRICS':^80}")
    print("="*80)

    # Calculate downside deviation
    negative_returns = portfolio_returns[portfolio_returns < 0]
    downside_deviation = negative_returns.std() * np.sqrt(252)
    sortino_ratio = (annualized_return - risk_free_rate) / downside_deviation if downside_deviation > 0 else np.nan

    # Calculate win rate
    winning_days = (portfolio_returns > 0).sum()
    total_days = len(portfolio_returns)
    win_rate = winning_days / total_days

    print(f"\n📊 Sortino Ratio: {sortino_ratio:.3f}")
    print(f"✅ Win Rate: {win_rate*100:.2f}% ({winning_days}/{total_days} days)")
    print(f"📈 Average Win: {portfolio_returns[portfolio_returns > 0].mean()*100:.2f}%")
    print(f"📉 Average Loss: {portfolio_returns[portfolio_returns < 0].mean()*100:.2f}%")

    # Correlation matrix
    print("\n" + "="*80)
    print(f"{'CORRELATION MATRIX':^80}")
    print("="*80)

    corr_matrix = stock_returns.corr()
    print("\n", corr_matrix.round(3))

    # Summary comparison with S&P 500
    print("\n" + "="*80)
    print(f"{'BENCHMARK COMPARISON (S&P 500)':^80}")
    print("="*80)

    spy_data = yf.download('SPY', start=start_date, end=end_date, progress=False)['Close']
    spy_return = (spy_data.iloc[-1] - spy_data.iloc[0]) / spy_data.iloc[0]
    spy_ann_return = (1 + spy_return) ** (252 / len(spy_data)) - 1

    print("\n" + "="*80)
    print(f"{'BENCHMARK COMPARISON (S&P 500)':^80}")
    print("="*80)

    try:
        spy_data = yf.download('SPY', start=start_date, end=end_date, progress=False)['Close']

        # Extract scalar values properly
        if isinstance(spy_data, pd.Series):
            spy_start = spy_data.iloc[0]
            spy_end = spy_data.iloc[-1]
        else:
            spy_start = float(spy_data.iloc[0])
            spy_end = float(spy_data.iloc[-1])

        spy_return = (spy_end - spy_start) / spy_start
        spy_ann_return = (1 + spy_return) ** (252 / len(spy_data)) - 1

        print(f"\n🏦 S&P 500 Return: {spy_return*100:.2f}%")
        print(f"📊 Portfolio Return: {total_return*100:.2f}%")
        print(f"🎯 Alpha (vs S&P 500): {(total_return - spy_return)*100:.2f}%")
    except Exception as e:
        print(f"\n⚠️ Unable to fetch S&P 500 data for comparison: {e}")

    print("\n" + "="*80 + "\n")

    return {
        'portfolio_value': portfolio_value,
        'returns': portfolio_returns,
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'individual_returns': individual_returns
    }

# AUTOMATICALLY EXTRACT TICKERS FROM RESULT
portfolio_tickers = extract_portfolio_from_result(result)
print(f"🔍 Automatically extracted portfolio: {', '.join(portfolio_tickers)}")

# Extract potential gains from the analysis to create custom weights
def extract_weights_from_potential(crew_output):
    """Extract weights based on potential gains mentioned in the analysis"""

    raw_text = crew_output.raw

    # Extract potential gains for each ticker
    potentials = {}

    # Pattern to find "TICKER: ... potential increase of X%"
    pattern = r'([A-Z]{2,5}):[^:]*?potential increase of ([\d.]+)%'
    matches = re.findall(pattern, raw_text, re.IGNORECASE)

    for ticker, potential in matches:
        potentials[ticker] = float(potential)

    # If we found potentials, create weights proportional to them
    if potentials and len(potentials) >= len(portfolio_tickers) * 0.7:  # At least 70% of tickers
        # Normalize potentials to create weights
        total_potential = sum(potentials.values())
        weights = []

        for ticker in portfolio_tickers:
            if ticker in potentials:
                weight = potentials[ticker] / total_potential
            else:
                weight = 1 / len(portfolio_tickers)  # Equal weight for missing
            weights.append(weight)

        # Normalize weights to sum to 1
        weights = np.array(weights)
        weights = weights / weights.sum()

        return weights

    return None  # Return None for equal weights

# Run backtest for different time periods
# Example 1: Last 6 months with equal weights
print("\n\n🔍 BACKTEST SCENARIO 1: Last 6 Months (Equal Weight)")
results_6m = backtest_portfolio(
    tickers=portfolio_tickers,
    start_date='2024-07-01',
    end_date='2024-12-31',
    initial_capital=100000
)

# Example 2: Last 1 year with equal weights
print("\n\n🔍 BACKTEST SCENARIO 2: Last 1 Year (Equal Weight)")
results_1y = backtest_portfolio(
    tickers=portfolio_tickers,
    start_date='2024-01-01',
    end_date='2024-12-31',
    initial_capital=100000
)

# Example 3: Custom weights based on potential gains from analysis
custom_weights = extract_weights_from_potential(result)
if custom_weights is not None:
    print("\n\n🔍 BACKTEST SCENARIO 3: Last 3 Months (Weighted by Potential)")
    print(f"📊 Extracted weights based on potential gains:")
    for ticker, weight in zip(portfolio_tickers, custom_weights):
        print(f"   {ticker}: {weight*100:.1f}%")

    results_weighted = backtest_portfolio(
        tickers=portfolio_tickers,
        start_date='2024-10-01',
        end_date='2024-12-31',
        initial_capital=100000,
        weights=custom_weights
    )
else:
    # Fallback to manual weights if extraction fails
    print("\n\n🔍 BACKTEST SCENARIO 3: Last 3 Months (Manual Strategic Weights)")
    # Create strategic weights (higher for Strong Buy and high potential)
    manual_weights = [1/len(portfolio_tickers)] * len(portfolio_tickers)
    results_weighted = backtest_portfolio(
        tickers=portfolio_tickers,
        start_date='2024-10-01',
        end_date='2024-12-31',
        initial_capital=100000,
        weights=manual_weights
    )

# Quick comparison function
def compare_scenarios(scenarios):
    """Compare multiple backtest scenarios"""
    print("\n" + "="*80)
    print(f"{'SCENARIO COMPARISON':^80}")
    print("="*80)
    print(f"\n{'Scenario':<20} {'Total Return':<15} {'Sharpe Ratio':<15} {'Max Drawdown':<15}")
    print("-" * 65)

    for name, result in scenarios:
        print(f"{name:<20} {result['total_return']*100:<14.2f}% {result['sharpe_ratio']:<14.3f} {result['max_drawdown']*100:<14.2f}%")

# Compare all scenarios
compare_scenarios([
    ("6 Months", results_6m),
    ("1 Year", results_1y),
    ("3M Weighted", results_weighted)
])

🔍 Automatically extracted portfolio: DAVA, AKAM, SENS, NICE, SABR, GLOB, ADBE


🔍 BACKTEST SCENARIO 1: Last 6 Months (Equal Weight)

                          PORTFOLIO BACKTEST ANALYSIS                           

📅 Period: 2024-07-01 to 2024-12-31
💰 Initial Capital: $100,000.00
📊 Portfolio: DAVA, AKAM, SENS, NICE, SABR, GLOB, ADBE

⏳ Downloading historical data...
Price            Close                                                      \
Ticker            ADBE       AKAM       DAVA        GLOB        NICE  SABR   
Date                                                                         
2024-07-01  560.010010  89.529999  29.100000  176.289993  168.600006  2.57   
2024-07-02  567.710022  90.820000  29.959999  176.550003  169.539993  2.58   
2024-07-03  570.150024  90.330002  30.000000  176.399994  169.889999  2.70   

Price                    High                        ...        Open        \
Ticker       SENS        ADBE       AKAM       DAVA  ...        NICE  SABR   
Date  